In [1]:
# Preparation
from acquire_msc import get_all_readme_files_and_languages
# from acquire_readme_files import temp_get_all_readme_files_and_languages
import unicodedata
import re
import json

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords

import pandas as pd

In [2]:
url_list = ['microsoft/Terminal', 'jackfrued/Python-100-Days', 
            'kkuchta/css-only-chat', 'microsoft/PowerToys', 'jolaleye/cssfx', 
            'MisterBooo/LeetCodeAnimation', 'flutter/flutter_web', 
            'TheAlgorithms/Python', 'hiroppy/fusuma', 'CyC2018/CS-Notes', 
            'jaywcjlove/linux-command', 'flutter/flutter', '996icu/996.ICU', 
            'STVIR/pysot', 'minamarkham/formation', 'azl397985856/leetcode', 
            'qianguyihao/Web', 'react-native-windows', 'sql-machine-learning/sqlflow', 
            'sabakkps/backslide', 'dgryski/go-perfbook', 'Snailclimb/JavaGuide', 
            'microsoft/vscode', 'markphelps/flipt', 'teoga/awesome-product-design']

In [3]:
def basic_clean(article):
    '''
    take in a string (article) and return it after applying some basic text cleaning to it:
        - lowercase everything
        - normalize unicode characters
        - replace anything that is not a letter, number, whitespace or a single quote
    '''
    new_article = article.lower()
    new_article = re.sub(r'\s', ' ', new_article)
    normalized = unicodedata.normalize('NFKD', new_article)                .encode('ascii', 'ignore')                .decode('utf-8')
    without_special_chars = re.sub(r'[^\w\s]', ' ', normalized)
    word_list = without_special_chars.split()
    word_list = ' '.join(word_list)
    return word_list

In [4]:
def tokenize(article):
    '''tokenize all the words in the string, article'''
    tokenizer = nltk.tokenize.ToktokTokenizer()
    new_article = tokenizer.tokenize(article, return_str=True)
    return new_article

In [5]:
def print_stop_words(article):
    '''accept some text, apply stemming to all of the words,
        and print a list of value counts for all the stemmed words'''
    # Create the nltk stemmer object, then use it
    ps = nltk.porter.PorterStemmer()
    stems = [ps.stem(word) for word in article.split()]
    print(pd.Series(stems).value_counts())

In [6]:
def stem(article):
    '''accept a string and return it after applying stemming to all the words'''
    ps = nltk.stem.PorterStemmer()
    article_stemmed = ''.join([ps.stem(word) for word in article])
    return article_stemmed

In [7]:
def lemmatize(article):
    '''accept a string and return it after applying lemmatization to each word.'''
    wnl = nltk.stem.WordNetLemmatizer()
    lemmatized_words = [wnl.lemmatize(word) for word in article]
    article_lemmatized = ''.join(lemmatized_words)
    return article_lemmatized

In [8]:
def remove_stopwords(article, extra_words = None, exclude_words = None):
    '''remove all the stopwords, including all the words in extra_words and excluding
    all the words in exclude list'''

    # get basic stopword list
    stopword_list = stopwords.words('english')

    # add extra words    
    if extra_words != None:
        stopword_list = stopword_list + extra_words
    # remove excluded words
    if exclude_words != None:
        stopword_list = [word for word in stopword_list if word not in exclude_words]
    
    without_stopwords = [word for word in article.split(' ') if word not in stopword_list]
    article_without_stopwords = ' '.join(without_stopwords)
    return article_without_stopwords

In [9]:
def prep_article(this_dict, extra_words = None, exclude_words = None):
    '''
    takes in a dictionary representing an article and returns a dictionary that 
    looks like this:
            {
             'title': 'the original title',
             'original': original,
             'stemmed': article_stemmed,
             'lemmatized': article_lemmatized,
             'clean': article_without_stopwords
            }
    Note that if the orignal dictionary has a title property, it will remain unchanged 
    (same goes for the category property).
    '''
    # put the content section into article and make a copy
#     article = articles['content'][article_index]    # needed for 
    article = this_dict['content']
    original = article

    '''
    apply some basic text cleaning to the string, article:
        - lowercase everything
        - normalize unicode characters
        - replace anything that is not a letter, number, whitespace or a single quote
    '''
    article = basic_clean(article)

    '''tokenize all the words in the string, article'''
    article = tokenize(article)

    '''applying stemming to all the words in the string, article'''
    article_stemmed = stem(article)
    
    ''''apply lemmatization to each word in the string, article'''
    article_lemmatized = lemmatize(article)
    
    '''remove all the stopwords, including all the words in extra_words and excluding
    all the words in exclude list'''
    article_without_stopwords = remove_stopwords(article, extra_words, exclude_words)

    keys = list(this_dict.keys())
    
    new_dict = {
         'title': this_dict['title'],
         'original': original,
         'category': [this_dict['category'] if 'category' in keys else 'blog'],
         'stemmed': article_stemmed,
         'lemmatized': article_lemmatized,
         'clean': article_without_stopwords
        }
    return new_dict

In [10]:
def prepare_article_data(articles, extra_words = None, exclude_words = None):
    # takes in the list of articles dictionaries, 
    # applies the prep_article function to each one, 
    # and returns the transformed data.
    transformed_articles = []

    for article_index in range(len(articles)):
        transformed_entry = prep_article(articles[article_index], extra_words, exclude_words)
        transformed_articles.append(transformed_entry.copy())

    return transformed_articles

In [11]:
def prep_news_article(this_dict, extra_words = None, exclude_words = None):
    '''
    takes in a dictionary representing an article and returns a dictionary that 
    looks like this:
            {
             'title': 'the original title',
             'original': original,
             'stemmed': article_stemmed,
             'lemmatized': article_lemmatized,
             'clean': article_without_stopwords
            }
    Note that if the orignal dictionary has a title property, it will remain unchanged 
    (same goes for the category property).
    '''
    # put the content section into article and make a copy
#     article = articles['content'][article_index]    # needed for 
    article = this_dict['content']
    original = article

    '''
    apply some basic text cleaning to the string, article:
        - lowercase everything
        - normalize unicode characters
        - replace anything that is not a letter, number, whitespace or a single quote
    '''
    article = basic_clean(article)

    '''tokenize all the words in the string, article'''
    article = tokenize(article)

    '''applying stemming to all the words in the string, article'''
    article_stemmed = stem(article)
    
    ''''apply lemmatization to each word in the string, article'''
    article_lemmatized = lemmatize(article)
    
    '''remove all the stopwords, including all the words in extra_words and excluding
    all the words in exclude list'''
    article_without_stopwords = remove_stopwords(article, extra_words, exclude_words)

    keys = list(this_dict.keys())
    
    new_dict = {
         'title': this_dict['title'],
         'original': original,
         'category': [this_dict['category'] if 'category' in keys else 'blog'],
         'stemmed': article_stemmed,
         'lemmatized': article_lemmatized,
         'clean': article_without_stopwords
        }
    return new_dict

In [12]:
def prepare_news_article_data(articles, extra_words = None, exclude_words = None):
    # takes in the list of articles dictionaries, 
    # applies the prep_article function to each one, 
    # and returns the transformed data.
    transformed_articles = []
    for article_index in range(len(articles)):
        transformed_entry = prep_news_article(articles[article_index], extra_words, exclude_words)
        transformed_articles.append(transformed_entry.copy())

    return transformed_articles

In [14]:
# if __name__ == "__main__":
'''create a list of extra words and another of words to exclude from the stoplist'''
extra_words = ['codeup']
exclude_words = ['']

articles = get_all_readme_files_and_languages(url_list)
print()
transformed_data = prepare_news_article_data(articles, extra_words, exclude_words)
for index in range(len(transformed_data)):
    print ('index:', index)
    for key in transformed_data[index]:
        print('key:', key)
        print(transformed_data[index][key])
        print('*******************')
    print ('$$$$$$$$$$$$$$$$$$$$$$$$$$$$$')


index: 0
key: title
microsoft/Terminal
*******************
key: original
Welcome! This repository contains the source code for:

Windows Terminal
The Windows console host (conhost.exe)
Components shared between the two projects
ColorTool
Sample projects that show how to consume the Windows Console APIs

Build Status



Project
Build Status




Terminal



ColorTool




Terminal & Console Overview
Please take a few minutes to review the overview below before diving into the code:
Windows Terminal
Windows Terminal is a new, modern, feature-rich, productive terminal application for command-line users. It includes many of the features most frequently requested by the Windows command-line community including support for tabs, rich text, globalization, configurability, theming & styling, and more.
The Terminal will also need to meet our goals and measures to ensure it remains fast, and efficient, and doesn't consume vast amounts of memory or power.
The Windows console host
The Windows conso